<a href="https://colab.research.google.com/github/nang-e/10000recipe/blob/main/recipe_code/%E1%84%86%E1%85%A1%E1%86%AB%E1%84%80%E1%85%A2%E1%84%8B%E1%85%B4%E1%84%85%E1%85%A6%E1%84%89%E1%85%B5%E1%84%91%E1%85%B5_%E1%84%8C%E1%85%A2%E1%84%85%E1%85%AD%E1%84%8B%E1%85%B5%E1%86%B8%E1%84%85%E1%85%A7%E1%86%A8%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from IPython.core.interactiveshell import InteractiveShell
import sys
InteractiveShell.ast_node_interactivity='all'

In [ ]:
recipe=pd.read_excel('만개의레시피_레시피명,재료_전처리.xlsx')

In [ ]:
recipe = recipe.drop(146,axis = 0)

In [ ]:
recipe['전처리재료'].unique()

array(['[]', "['소금', '홍고추']",
       "['설탕[토핑]메이플시럽', '시나몬가루', '연유', '볶은콩가루', '바닐라아이스크림', '견과류']", ...,
       "['돼지고기', '배추김치', '김치국물', '두부', '양파', '들기름', '고추가루', '디포리', '무', '물']",
       "['닭안심', '우유', '빵가루', '밀가루', '계란', '2', '소금', '후추', '맛술', '2']",
       "['등갈비', '김치', '월계수잎', '설탕']"], dtype=object)

In [ ]:
time=[]
size=[]
level=[]
time=recipe['소요시간'].unique()
size=recipe['인분'].unique()
level=recipe['난이도'].unique()

time, size, level

(array(['20분 이내', '2시간 이상', '15분 이내', '30분 이내', '60분 이내', '120분 이내',
        '10분 이내', '90분 이내', '5분 이내'], dtype=object),
 array(['3인분', '2인분', '1인분', '5인분', '6인분 이상', '4인분'], dtype=object),
 array(['아무나', '초급', '중급', '신의경지', '고급'], dtype=object))

In [ ]:
recipe.columns # 컬럼명 확인

Index(['레시피명', 'Index', '레시피링크', '소요시간', '인분', '난이도', '재료', '전처리재료'], dtype='object')

In [ ]:
recipe_c = recipe.copy() # 복사본 생성

## 전처리

In [ ]:
def preprocess():
    # 시간 분단위로 통일
    recipe_c['소요시간'] = recipe_c['소요시간'].apply(\
                        lambda x: '121분 이상' if x == '2시간 이상' else x)
    # 난이도 범주형->숫자로 변환 
    ## 아무나: 1, 초급: 2, 중급: 3, 고급: 4, 신의경지: 5
    recipe_c['난이도'] = recipe_c['난이도'].apply(lambda x: 1 if x=='아무나'
                                       else 2 if x=='초급'
                                       else 3 if x=='중급'
                                       else 4 if x=='고급'
                                       else 5)
    # 인분과 소요시간에서 숫자만 추출
    recipe_c['인분'] = recipe_c['인분'].str.extract('([0-9])')
    recipe_c['소요시간'] = recipe_c['소요시간'].str.extract('(\d+)')

In [ ]:
preprocess()

In [ ]:
recipe_c.head()

,레시피명,Index,레시피링크,소요시간,인분,난이도,재료,전처리재료
0,제육 삼각 김밥,6963999,https://www.10000recipe.com/recipe/6963999,20,3,1,"[재료(약7개분량)]볶은제육불고기200g[밥양념]밥5공기,통깨2큰술,참기름2큰술,소...",[]
1,마약계란장 만들기 반숙계란장 레시피,6963998,https://www.10000recipe.com/recipe/6963998,121,2,2,"[재료]계란10개,소금,식초약간,양파1/2개,쪽파반줌,홍고추,청양고추각1개[양념장재...","['소금', '홍고추']"
2,크로와상 생지로 크로플 만들기 겉은 바삭 속은 쫄깃해요,6963996,https://www.10000recipe.com/recipe/6963996,15,2,1,"[재료]크로와상냉동생지4개,오일(식용유or버터),설탕[토핑]메이플시럽,시나몬가루,연...","['설탕[토핑]메이플시럽', '시나몬가루', '연유', '볶은콩가루', '바닐라아이..."
3,고기육수를 내서 깊고 푸짐한 두부김치찌개 간단하게 만들기,6963995,https://www.10000recipe.com/recipe/6963995,30,2,2,"[재료]묵은지적당량,돼지고기100g,설탕(생략가능)0.3T,다진마늘1T,두부적당량,...",[]
4,계란샌드위치 만들기 에그샌드위치 레시피,6963994,https://www.10000recipe.com/recipe/6963994,20,2,2,"[재료]계란6개,식빵6장,마요네즈3T,플레인요거트1T듬뿍,꿀약간,홀그레인머스타드약간...",[]


In [ ]:
# recipe_c : 단순 복사본
# recipe_c2 : 입력된 재료를 가진 행만 저장된 df
# recipe_c3 : 입력된 재료 및 특정 난도를 가진 행만 저장된 df
# recipe_c4 : 입력된 재료, 특정 난도 및 특정 수준 이하의 소요시간을 가진 행만 저장된 df
# recipe_c5 : 입력된 재료, 특정 난도 및 특정 수준 이하의 소요시간 및 특정 인분을 가진 행만 저장된 df

recipe_view=recipe_c.drop('전처리재료',axis = 1) # recipe_view: 전처리재료 열 제외한 df

## user_choice()<br>
- 사용자가 지닌 재료 입력 시 특정 난도, 소요시간, 인분에 따른 결과가 출력되고 사용자가 index를 고르는 방식

In [ ]:
# 재료 선택
def user_choice():
    recipe_c2=[]
    print('가지고 계신 재료를 입력하세요')
    recipe_name=input('사용자가 가진 재료를 입력하시오: ').split(',')

    for i in range(len(recipe_name)):
        if i==0:
            if recipe_c['재료'].str.contains(recipe_name[i]).value_counts()[0] == len(recipe_c):
                 print('해당 재료를 포함한 레시피가 존재하지 않습니다.')
                 sys.exit('종료합니다. 다시 입력해주세요.')
                 break
                 
        
            else:
            # 입력된 재료를 가진 레시피 행들은 recipe_c2에 저장
                recipe_c2 =recipe_c[recipe_c['재료'].str.contains(recipe_name[i])]
                if len(recipe_c2) == 0 :
                    print('해당 재료를 포함한 레시피가 존재하지 않습니다.')
                    sys.exit('종료합니다. 다시 입력해주세요.')
                    break
                print('*')
    
    
        else:
            if recipe_c2['재료'].str.contains(recipe_name[i]).value_counts()[0] == len(recipe_c):
                 print('해당 재료를 포함한 레시피가 존재하지 않습니다.')
                 sys.exit('종료합니다. 다시 입력해주세요.')
                 break
        
            else:
            # 입력된 재료를 가진 레시피 행들은 recipe_c2에 저장
                a =recipe_c2['재료'].str.contains(recipe_name[i])
                recipe_c2=recipe_c2[a] 
                if len(recipe_c2) == 0 :
                    print('해당 재료를 포함한 레시피가 존재하지 않습니다.')
                    sys.exit('종료합니다. 다시 입력해주세요.')
                    break
                print()
                print('*')
                print()
            

        # 레시피 난이도 입력
    recipe_level = input('난도를 몇으로 설정하시겠습니까? \
                        \n(단, 1~5까지의 숫자를 입력하시오.\
                        \n혹은 "상관 없음"이나 0 입력)": ')
    level_tf = recipe_c2['난이도'] == int(recipe_level) if recipe_level in ['1','2','3','4','5'] \
                                                    else recipe_c2['난이도'] == recipe_level

    if recipe_level == '상관없음' or recipe_level == '상관 없음' or recipe_level == '0':
        recipe_c3 = recipe_c2.copy() # 상관없음 입력시 입력 재료를 포함한 모든 레시피
        if len(recipe_c3) == 0:
            print('해당 난도의 레시피가 존재하지 않습니다.')
            sys.exit('종료합니다. 다시 입력해주세요.')
    elif level_tf.value_counts()[0] == len(recipe_c2):
        print('해당 난도의 레시피가 존재하지 않습니다.') # 입력 재료를 가진 난도의 레시피가 없을 경우
        sys.exit('종료합니다. 다시 입력해주세요')
    else:
        recipe_c3 = recipe_c2[recipe_c2['난이도'] == int(recipe_level)]
        if len(recipe_c3) == 0:
            print('해당 난도의 레시피가 존재하지 않습니다.')
            sys.exit('해당 난도의 레시피가 존재하지 않아 종료')



    print('---------------------------------------------------------------------------')


# 레시피 소요시간 입력
# ['20분 이내' '2시간 이상' '15분 이내' '30분 이내' '60분 이내' '120분 이내' '10분 이내' '90분 이내','5분 이내']
    recipe_time = input('몇 분 이하의 소요시간을 필요로 하시나요?: \
                        \n(단, 5 이상의 숫자를 입력하시오)) ')
    try:
        if int(recipe_time) < 5: 
            print(f'{recipe_time}분 이하의 레시피는 존재하지 않습니다.')
            sys.exit('해당 소요시간의 레시피가 존재하지 않아 종료합니다.')
            
        elif int(recipe_time) <= 120: 
            recipe_c4 = recipe_c3[recipe_c3['소요시간'].astype('int') <= int(recipe_time)]
            if len(recipe_c4) == 0:
                print('해당 소요시간의 레시피가 존재하지 않습니다.')
                sys.exit('종료합니다. 다시 입력해주세요.')
                
        elif int(recipe_time) >= 120: # 2시간 이상인 경우
            recipe_c4 = recipe_c3[recipe_c3['소요시간'].astype('int') == 121]
            if len(recipe_c4) == 0:
                print('해당 소요시간의 레시피가 존재하지 않습니다.')
                sys.exit('종료합니다. 다시 입력해주세요.')
                
    except (ValueError, TypeError):
        print('잘못 입력하셨습니다')
        sys.exit('해당 소요시간의 레시피가 존재하지 않아 종료합니다.')
    
    print()
    print('---------------------------------------------------------------------------')
    print()

    # 레시피 인분 입력
    recipe_n = input('몇 인분의 양이 필요하신가요?: ')
    recipe_n = re.sub('[^0-9]','',recipe_n)
    try:
        if int(recipe_n) <= 5:
            recipe_c5 = recipe_c4[recipe_c4['인분'].astype('int') == int(recipe_n)]
            if len(recipe_c5) == 0:
                print('해당 인분의 레시피가 존재하지 않습니다')
                sys.exit('종료합니다. 다시 검색해주세요')
                
        elif int(recipe_n) >= 6: # 6이상의 숫자를 입력한 경우 6인분 이상만 보이게
            recipe_c5 = recipe_c4[recipe_c4['인분'].astype('int') == 6]
            if len(recipe_c5) == 0:
                print('해당 인분의 레시피가 존재하지 않습니다')
                sys.exit('종료합니다. 다시 검색해주세요')
            
    except (ValueError, TypeError):
        print('잘못 입력하셨습니다.')
        sys.exit('해당 인분의 레시피가 존재하지 않아 종료합니다.')
    print()
    print('---------------------------------------------------------------------------')    
    print()
    
    #사용자 선택에 따른 레시피 출력
    print('난이도: '+str(recipe_level)+', 소요시간: '+str(recipe_time)+'분, 양: '+ str(recipe_n) +'인 요리를 검색합니다......' )
    print()
    print('총 '+ str(len(recipe_c5))+'개의 레시피가 존재합니다.')
    food=recipe_c5['전처리재료']
    if len(recipe_c5) == 0:
        sys.exit('해당하는 레시피가 존재하지 않습니다.\n\t\t다시 검색해주세요.')
    elif len(recipe_c5) >= 11:
        print('*')
        print('*')
        print('*')
        print('해당하는 레시피가 많아 일부 레시피만 로딩중')
        print('*')
        print('*')
        print('*')
        i = 1
        while i == 1:
            recipe_c6 = recipe_c5.reset_index()
            rand_n = np.random.choice(len(recipe_c6), 10, replace = False)
            rand_n.sort()
            for i in rand_n:
                print(recipe_c6.index[i],'    ', recipe_c6['레시피명'][i])
            print('------------------------------------------------------')
            print('자세히 보고 싶은 레시피 번호를 입력하세요')
            print('다른 결과를 보고 싶다면 다른 문자를 입력하세요.')
            choice_index = input()
            try:
                choice_index = int(choice_index)
                print()
                print()
                if recipe_c6.iloc[choice_index]['index'] <= 145:
                    print('레시피명: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],0])
                    print('레시피링크: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],2])
                    print('⬈⬈⬈ 링크 클릭 시 사이트로 이동 ﹒﹒﹒')
                    print('소요시간: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],3])
                    print('인분: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],4])
                    print('난이도: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],5])
                    print('재료: ', recipe.iloc[recipe_c6.iloc[choice_index]['index'],6])
                    i = 0
                else:
                    print('레시피명: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,0])
                    print('레시피링크: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,2])
                    print('⬈⬈⬈ 링크 클릭 시 사이트로 이동 ﹒﹒﹒')
                    print('소요시간: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,3])
                    print('인분: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,4])
                    print('난이도: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,5])
                    print('재료: ', recipe.iloc[recipe_c6.iloc[choice_index]['index']-1,6])
                    i = 0

            except ValueError:
                print('*')
                print('*')
                print('*')
                print('다시 검색 중입니다')
                print('*')
                print('*')
                print('*')
                i = 1

    else:
        print(recipe_c5['레시피명'])
        print('------------------------------------------------------')
        print('자세히 보고 싶은 레시피 번호를 입력하세요')
        choice_index=int(input())
        print()
        print()
        if choice_index <= 145:
            print('레시피명: ', recipe.iloc[choice_index,0])
            print('레시피링크: ', recipe.iloc[choice_index,2])
            print('⬈⬈⬈ 링크 클릭 시 사이트로 이동 ﹒﹒﹒')
            print('소요시간: ', recipe.iloc[choice_index,3])
            print('인분: ', recipe.iloc[choice_index,4])
            print('난이도: ', recipe.iloc[choice_index,5])
            print('재료: ', recipe.iloc[choice_index,6])
        else:
            print('레시피명: ', recipe.iloc[choice_index-1,0])
            print('레시피링크: ', recipe.iloc[choice_index-1,2])
            print('⬈⬈⬈ 링크 클릭 시 사이트로 이동 ﹒﹒﹒')
            print('소요시간: ', recipe.iloc[choice_index-1,3])
            print('인분: ', recipe.iloc[choice_index-1,4])
            print('난이도: ', recipe.iloc[choice_index-1,5])
            print('재료: ', recipe.iloc[choice_index-1,6])
    
    return recipe_name,food

In [ ]:
recipe_name, food=user_choice()

가지고 계신 재료를 입력하세요
사용자가 가진 재료를 입력하시오: 김치,계란,참치
*

*


*

난도를 몇으로 설정하시겠습니까?                         
(단, 1~5까지의 숫자를 입력하시오.                        
혹은 "상관 없음"이나 0 입력)": 2
---------------------------------------------------------------------------
몇 분 이하의 소요시간을 필요로 하시나요?:                         
(단, 5 이상의 숫자를 입력하시오)) 10

---------------------------------------------------------------------------

몇 인분의 양이 필요하신가요?: 1

---------------------------------------------------------------------------

난이도: 2, 소요시간: 10분, 양: 1인 요리를 검색합니다......

총 2개의 레시피가 존재합니다.
5956     요즘 핫한 접어 먹는 김밥   가장 맛있는 조합 접는김밥
60166                파김치 비빔밥 완전 별미 한그릇음식
Name: 레시피명, dtype: object
------------------------------------------------------
자세히 보고 싶은 레시피 번호를 입력하세요
5956


레시피명:  요즘 핫한 접어 먹는 김밥   가장 맛있는 조합 접는김밥
레시피링크:  https://www.10000recipe.com/recipe/6953277
⬈⬈⬈ 링크 클릭 시 사이트로 이동 ﹒﹒﹒
소요시간:  10분 이내
인분:  1인분
난이도:  초급
재료:  [밥간하기],밥,1공기,참기름,1스푼,맛소금,1꼬집,깨,2스푼,[참치접는김밥],참치,2스푼,깻잎,2장,스팸,2장,쌈무,2장,마요네즈,2스푼,계란,1개,[김치접는김밥],김치,1줌,스팸

## need_ingredient()<br>
- 사용자가 일부 재료를 가지고 있을 때 무슨 재료가 추가로 더 필요한 지 알려주는 코드
- 사용자가 선택한 기준(재료, 난이도, 시간 등)에 맞는 레시피들의 재료 중에서 가장 많이 나온 재료를 출력

In [ ]:
def need_ingredient():
    counts={}
# 재료 분리해서 횟수 세기 
    
    for line in food:
        words=line.split(',')
        for word in words:
            counts[word]=counts.get(word,0)+1

        
    x=counts.items()
    sort_food= sorted(x, key=lambda x: x[1], reverse=True)

    
    #사용자가 입력한 재료가 sort_food 리스트에 있는지 확인하고 삭제 (추가 재료만 필요하니까)
    num=[]
    recipe_name
    for i in range(len(recipe_name)):
        for j in range(len(sort_food)):
            if recipe_name[i] in sort_food[j][0] :
                num.append(j)

    num.sort()
    num
    y=len(num)
    
    
    for x in range(y):
        z=num[0]
        del sort_food[z]
        num.remove(z)
        for j in range(len(num)):
            num[j]= num[j]-1
    
    # 상위 3개 재료만 출력
    best_3=sort_food[0:3]

    print('입력하신 재료: ',recipe_name)
    print('>>>저희가 보유한 레시피 분석 결과, 추가로 필요한 재료 best3은 ', best_3,'입니다.')
    

In [ ]:
need_ingredient()

입력하신 재료:  ['김치', '계란', '참치']
>>>저희가 보유한 레시피 분석 결과, 추가로 필요한 재료 best3은  [(" '스팸'", 2), ("['[밥간하기]'", 1), (" '밥'", 1)] 입니다.
